# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')


# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avarua,-21.2078,-159.7750,22.03,60,20,2.57,CK,2024-06-08
1,1,port-aux-francais,-49.3500,70.2167,6.42,98,100,13.24,TF,2024-06-08
2,2,roma,43.2128,-75.4557,14.42,94,75,0.89,US,2024-06-08
3,3,adamstown,-25.0660,-130.1015,22.34,77,19,5.28,PN,2024-06-08
4,4,jackson,42.4165,-122.8345,20.93,54,0,1.54,US,2024-06-08


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", 
)

# Display the map
hvplot.save(humidity_map, "output_data/humidity_map.html")
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
cities_narrow_lats_df = city_data_df[(city_data_df['Lat'] >= -19) & (city_data_df['Lat'] <= 40)]

# Drop any rows with null values
ideal_weather_city_df = cities_narrow_lats_df.dropna()

# Display sample data
ideal_weather_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,riachao das neves,-11.7461,-44.9100,17.25,60,34,0.30,BR,2024-06-08
13,13,palikir - national government center,6.9248,158.1611,26.41,89,100,1.54,FM,2024-06-08
14,14,lihue,21.9789,-159.3672,27.58,84,20,6.17,US,2024-06-08
15,15,danli,14.0333,-86.5833,20.60,98,98,1.11,HN,2024-06-08
24,24,santa elena de uairen,4.6167,-61.1333,18.64,94,16,0.43,VE,2024-06-08
...,...,...,...,...,...,...,...,...,...,...
561,561,pisco,-13.7000,-76.2167,17.03,88,100,2.57,PE,2024-06-08
564,564,paita,-5.0892,-81.1144,18.40,87,65,5.39,PE,2024-06-08
566,566,mukalla,14.5425,49.1242,33.34,59,4,3.09,YE,2024-06-08
568,568,sola,-13.8833,167.5500,26.75,71,25,6.84,VU,2024-06-08


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city_df[['City', 'Country','Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
9,riachao das neves,BR,-11.7461,-44.9100,60,
13,palikir - national government center,FM,6.9248,158.1611,89,
14,lihue,US,21.9789,-159.3672,84,
15,danli,HN,14.0333,-86.5833,98,
24,santa elena de uairen,VE,4.6167,-61.1333,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
params = {
    "limit":limit,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
        
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
riachao das neves - nearest hotel: No hotel found
palikir - national government center - nearest hotel: Mangrove Bay Hotel
lihue - nearest hotel: Kauai Palms
danli - nearest hotel: Hotel granada
santa elena de uairen - nearest hotel: Posada Villa Fairmont
andkhoy - nearest hotel: No hotel found
miri - nearest hotel: One Times One Inn
dunda - nearest hotel: No hotel found
dharmavaram - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
tucumcari - nearest hotel: Pow Wow Inn
chania - nearest hotel: Morum
qurayyat - nearest hotel: دانة الشمال
matsue - nearest hotel: サンラポーむらくも
carloforte - nearest hotel: Hotel Riviera
lamu - nearest hotel: Hotel Archipelago villa bed and breakfast
taoudenni - nearest hotel: No hotel found
aripuana - nearest hotel: No hotel found
hithadhoo - nearest hotel: Pebbles Inn
tongchuanshi - nearest hotel: No hotel found
pacasmayo - nearest hotel: Playa Linda
tamandare - nearest hotel: Pousada Baia dos Corais
atafu village - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
9,riachao das neves,BR,-11.7461,-44.9100,60,No hotel found
13,palikir - national government center,FM,6.9248,158.1611,89,Mangrove Bay Hotel
14,lihue,US,21.9789,-159.3672,84,Kauai Palms
15,danli,HN,14.0333,-86.5833,98,Hotel granada
24,santa elena de uairen,VE,4.6167,-61.1333,94,Posada Villa Fairmont
...,...,...,...,...,...,...
561,pisco,PE,-13.7000,-76.2167,88,La Portada
564,paita,PE,-5.0892,-81.1144,87,Victor's
566,mukalla,YE,14.5425,49.1242,59,فندق الجابري
568,sola,VU,-13.8833,167.5500,71,No hotel found


In [19]:
# Export the Hotel into a csv
hotel_df.to_csv("output_data/hotel.csv", index_label="City_ID")

In [20]:
# Read saved data
hotel_df = pd.read_csv("output_data/hotel.csv", index_col="City_ID")

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
9,riachao das neves,BR,-11.7461,-44.9100,60,No hotel found
13,palikir - national government center,FM,6.9248,158.1611,89,Mangrove Bay Hotel
14,lihue,US,21.9789,-159.3672,84,Kauai Palms
15,danli,HN,14.0333,-86.5833,98,Hotel granada
24,santa elena de uairen,VE,4.6167,-61.1333,94,Posada Villa Fairmont


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Lat",
    scale = 1,
    color = "City", 
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hvplot.save(hotel_map, "output_data/hotel_map.html")
hotel_map 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)